HOMEWORK 2
ES 3.A

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

W = np.array([
    [0,   3/4, 3/4, 0,   0  ],
    [0,   0,   1/4, 1/4, 2/4],
    [0,   0,   0,   1,   0  ],
    [0,   0,   0,   0,   1  ],
    [0,   0,   0,   0,   0  ]
])


n_states = W.shape[0]
sink_node = n_states-1
o_node = 0

w = np.sum(W, axis=1)
# rate of d is 2
w[sink_node] = 2
D = np.diag(w)
# contruct the P matrix,  P = D^-1 * Lambda
P = np.linalg.inv(D) @ W

In [ ]:
import numpy as np
import sys
max_integer = sys.maxsize

# define simulaation parameters
n_simulations = 1
n_steps = 6000
end_time = 60

# rate of node lambda 100
rate_lambda = 100
lambda_node = n_states

return_times_list = []
tot_particles_in_node = np.zeros((n_states,n_simulations))
for simulation in range(n_simulations):

    # save for eaach node how many particles are there for each step
    particles_in_node = np.zeros((n_states,n_steps), dtype=int)
    # for each step save the transition time
    transition_times = np.zeros(n_steps)

    # initialize the time for the next transition for each node
    # max integer because the node is empty so it won't transition
    t_next = np.zeros(n_states+1)+max_integer
    # compute the waiting time to the next transition of node lambda
    # t_next = -log(u) / r_lambda
    t_next[-1] = -np.log(np.random.rand()) / rate_lambda

    for t in range(1,n_steps):
        # find which node is the next to transition
        min_time = min(t_next)
        node = np.where(t_next==min_time)[0][0]

        # save transition time
        transition_times[t] = transition_times[t-1] + t_next[node]

        # update particles in nodes
        particles_in_node[:, t] = particles_in_node[:, t-1]
        if node == lambda_node:
            # if lambda transitioned add a particle to o
            particles_in_node[o_node, t] += 1

        elif node == sink_node:
            # if d transitioned remove a particle from it
            particles_in_node[sink_node, t] -= 1
        else:
            # find destination node and update number of particles
            next_node = np.random.choice(n_states, p=P[node,:])
            particles_in_node[node, t] -= 1
            particles_in_node[next_node, t] += 1

        # compute for every node a new transition time
        t_next = np.zeros(n_states+1)+max_integer
        for node in range(n_states):
            rate = particles_in_node[node , t] * w[node]
            if rate != 0:
                t_next[node] = -np.log(np.random.rand()) / rate
        t_next[-1] = -np.log(np.random.rand()) / rate_lambda
        # stop after 60 time units
        if transition_times[t] > end_time:
            break

    tot_particles_in_node[:, simulation] = particles_in_node[:,t]

mean_particles_in_node = np.zeros(n_states)
mean_particles_in_node = [ np.mean(tot_particles_in_node[node,:]) for node in range(n_states) ]

print(f"Media particelle in ogni nodo su {n_simulations} simulazioni: {mean_particles_in_node}")

In [ ]:
# plot evolution of nodes during time
for node in range(n_states):
    plt.plot(particles_in_node[node,:])
    plt.title(f'node {node+1}')
    plt.show()

HOMEWORK 2
ES 3.B

In [ ]:
import numpy as np
import sys
max_integer = sys.maxsize

# define simulaation parameters
n_simulations = 10
n_steps = 6000
end_time = 60

# rate of lambda node 1
rate_lambda = 1
lambda_node = n_states

return_times_list = []
tot_particles_in_node = np.zeros((n_states,n_simulations))
for simulation in range(n_simulations):
    # save for eaach node how many particles are there for each step
    particles_in_node = np.zeros((n_states,n_steps), dtype=int)
    # for each step save the transition time
    transition_times = np.zeros(n_steps)

    # initialize the time for the next transition for each node
    # max integer because the node is empty so it won't transition
    t_next = np.zeros(n_states+1)+max_integer
    # compute the waiting time to the next transition of node lambda
    # t_next = -log(u) / r_lambda
    t_next[-1] = -np.log(np.random.rand()) / rate_lambda

    for t in range(1,n_steps):

        # find which node is the next to transition
        min_time = min(t_next)
        node = np.where(t_next==min_time)[0][0]

        # save transition time
        transition_times[t] = transition_times[t-1] + t_next[node]

        # compute the waiting time to the next transition
        # t_next = -log(u)/r
        # NOTE: we use the rate w[pos[t]] of the clock of the current position of each particle.
        # So it's a Local Clock
        t_next = np.zeros(n_states+1)+max_integer
        for n in range(n_states):
            if particles_in_node[n, t-1] > 0:
                t_next[n] = -np.log(np.random.rand()) / w[n]
        # compute the next transition time fro lambda
        t_next[-1] = -np.log(np.random.rand()) / rate_lambda

        # update particles in nodes
        particles_in_node[:, t] = particles_in_node[:, t-1]

        if node == lambda_node:
            # if lambda transitioned add a particle to o
            particles_in_node[o_node, t] += 1

        elif node == sink_node:
            # if d transitioned remove a particle from it
            particles_in_node[sink_node, t] -= 1

        else:
            # find next node and update particles in node
            next_node = np.random.choice(n_states, p=P[node,:])
            particles_in_node[node, t] -= 1
            particles_in_node[next_node, t] += 1
                # stop after 60 time units
        if transition_times[t] > end_time:
            break

    tot_particles_in_node[:, simulation] = particles_in_node[:,t]

mean_particles_in_node = np.zeros(n_states)
mean_particles_in_node = [ np.mean(tot_particles_in_node[node,:]) for node in range(n_states) ]

print(f"Media particelle in ogni nodo su {n_simulations} simulazioni: {mean_particles_in_node}")

In [ ]:
# plot evolution of nodes during time
for node in range(n_states):
    plt.plot(particles_in_node[node,:])
    plt.title(f'node {node}')
    plt.show()